In [355]:
import pandas as pd
import sqlite3 as db
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score,explained_variance_score
from sklearn.cluster import KMeans
from statsmodels.tsa.seasonal import seasonal_decompose

In [3]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [4]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [5]:
def stopQualityMask(speciesCode: str,closestStop: str):
    cnx = connectDB()
    try:
        gap = dt.date.today().year-2018
        query = f'SELECT COUNT(DISTINCT(year)) as "frq" FROM coefficients_bySpecies WHERE speciesCode = "{speciesCode}" AND closestStop = "{closestStop}"'
        coeficients = pd.read_sql(query,con=cnx)
        coeficients['frq'] = coeficients.apply(lambda g: (g.frq/gap),axis=1)
    except Exception as maskExc:
        raise maskExc
    return coeficients['frq']

In [224]:
def log_stopMetrics(speciesCode,key,testBlob,latestUpdate):
    cnx = connectDB()
    cur = cnx.cursor()
    try:
        sqliteInsert = """INSERT INTO stopMetricsBlob (speciesCode,key,testBlob,latestUpdate) VALUES (?,?,?,?);"""
        logTuple = (speciesCode,key,testBlob,latestUpdate)
        cur.execute(sqliteInsert,logTuple)
        cnx.commit()
        cur.close()
    except db.Error as sqlError:
        raise sqlError
    finally:
        if cnx:
            cnx.close()

In [274]:
#baseline request from the application layer
#outputs a list of birds at the stop and a classification based solely off the number of observations
def birdList_request(StopName: str,cnx):
    try:
        query = f'SELECT speciesCode,count(subId) as "checklists",(SELECT count(subId) FROM historicObservations hxobx WHERE hxobx.speciesCode=hsob.speciesCode) as "sightings" FROM historicObservations hsob LEFT JOIN closestStop on hsob.locId=closestStop.locId WHERE StopName = "{StopName}" GROUP BY speciesCode;'
        sightings = pd.read_sql(sql=query,con=cnx)
    #rareness at the stop
        sightings['stopGroup'] = int
        bucket = sightings['checklists'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['checklists'] <= bucket[0.15],'stopGroup'] = 1  #mythic
        sightings.loc[(sightings['checklists'] > bucket[0.15]) & (sightings['checklists'] <= bucket[0.5]),'stopGroup'] = 2  #rare
        sightings.loc[(sightings['checklists'] > bucket[0.5]) & (sightings['checklists'] < bucket[0.85]),'stopGroup'] = 3   #uncommon
        sightings.loc[(sightings['checklists'] >= bucket[0.85]) & (sightings['checklists'] <=bucket[1]),'stopGroup'] = 4    #common
    #overall rareness
        sightings['overall'] = int
        bucket = sightings['sightings'].quantile([0,0.15,0.5,0.85,1])
        sightings.loc[sightings['sightings'] <= bucket[0.15],'overall'] = 1
        sightings.loc[(sightings['sightings'] > bucket[0.15]) & (sightings['sightings'] <= bucket[0.5]),'overall'] = 2
        sightings.loc[(sightings['sightings'] > bucket[0.5]) & (sightings['sightings'] < bucket[0.85]),'overall'] = 3
        sightings.loc[(sightings['sightings'] >= bucket[0.85]) & (sightings['sightings'] <=bucket[1]),'overall'] = 4
        sightings['StopName'] = StopName
    #raise an exception if the stopName given is not valid and return a list of valid stop names
    except Exception as ex:
        raise ex
    return sightings


In [275]:
birdlist = birdList_request(StopName='EastSunsetWay',cnx=connectDB())

In [442]:
birdlist

,speciesCode,checklists,sightings,stopGroup,overall,StopName
0,amecro,91,293,4,4,EastSunsetWay
1,amedip,10,10,3,2,EastSunsetWay
2,amegfi,11,46,3,2,EastSunsetWay
3,amekes,1,1,1,1,EastSunsetWay
4,amerob,80,371,4,4,EastSunsetWay
...,...,...,...,...,...,...
100,wooduc,5,95,2,3,EastSunsetWay
101,x00051,1,1,1,1,EastSunsetWay
102,y00475,9,36,3,2,EastSunsetWay
103,yelwar,3,16,2,2,EastSunsetWay


For birds with robust data, do they behave the same at all stops? If not, what stops does each species appear to prefer? Is that preference explained by habitat?

In [483]:
#TODO #92 change over flat annual average to weekly average in wklyAbd_selectSpecies() and other calcs
def wklyAbd_selectSpecies(cnx,speciesList: list):
    cnx=cnx
    try:
        querySpecies = []
        for i in speciesList:
            i = str(i)
            querySpecies.append(i)
        querySpecies = str(querySpecies).strip('[]')
        #query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX  LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.locId in ({query_locIds})'
    #call baseline
        queryBaseline = f'SELECT speciesCode,obsDt,howMany FROM historicObservations WHERE speciesCode in ({querySpecies})'
        baseline = pd.read_sql(sql=queryBaseline,con=cnx,parse_dates=['obsDt'])
        baseline = baseline.assign(obsDt_week=baseline.obsDt.dt.isocalendar().week)
        baseline['howMany'].fillna(1,inplace=True)
        baselineAvg = baseline.groupby(['speciesCode'])['howMany'].mean()

        query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode in ({querySpecies});'
        obsData = pd.read_sql(query,con=cnx,parse_dates=['obsDt'])
        obsData = obsData.assign(obsDt_week=obsData.obsDt.dt.isocalendar().week)
        obsData['howMany'].fillna(1,inplace=True)
        obsData = obsData.groupby(['speciesCode','StopName','obsDt_week'])['howMany'].mean().reset_index()
    #compute relative abundance
        obsData['relativeAbundance'] = obsData.apply(lambda x: (x.howMany/baselineAvg[x.speciesCode]),axis=1)  #baseline relative abd, same process as Fink et. all
        #obsData['relativeAbundance'] = obsData.apply(lambda f: stopQualityMask(f.speciesCode,f.StopName)*f.relativeAbundance,axis=1)  #apply frequency mask
        #avgAbd = obsData.groupby(['speciesCode'])['relativeAbundance'].mean()
        #obsData['relativeAbundance'] = obsData.apply(lambda n: ((n.relativeAbundance)/(avgAbd[n.speciesCode])),axis=1)  #normalizing around average relative abundance
        obsData.sort_values(by=['StopName','obsDt_week'],ascending=True,inplace=True)
    except db.DatabaseError as dbExc:
        raise f'there was an issue with the database request: {dbExc}'
    except Exception as ex:
        raise ex
    finally: cnx.close()
    return obsData[['obsDt_week','speciesCode','StopName','relativeAbundance']]

Interpolation testing

In [492]:
gocks = wklyAbd_selectSpecies(cnx=connectDB(),speciesList=['gockin'])

In [680]:
for i in list([0.2,0.25,0.3,0.35,0.4,0.45,0.5]):
    i+1
i


0.5

                    #stop_obsData['sample'].copy
                    #stop_obsData = stop_obsData.assign(y_pred_name=stop_obsData['sample'].interpolate(method=test['method'],order=test['order'],limit_direction=test['limit_direction']).fillna(stop_obsData['relativeAbundance'])
                    stop_obsData[testName] = stop_obsData['sample'].interpolate(method=test['method'],order=test['order'],limit_direction=test['limit_direction']).fillna(stop_obsData['relativeAbundance'])
                    r_sqr = r2_score(y_true=stop_obsData['relativeAbundance'],y_pred=stop_obsData[testName])
                    #explVar = explained_variance_score(y_true=stop_obsData['relativeAbundance'],y_pred=stop_obsData['sample'])
                    if r_sqr < 0:
                        continue
                    else: blob = {'StopName':x,'method':test['method'],'order':test['order'],'sampleSize':v,'r_sqr':r_sqr}
                    testResults.append(blob)
        resultFrame = pd.DataFrame.from_dict(testResults)
        resultFrame = resultFrame.groupby(by=['StopName','method','order'])['r_sqr'].mean()
                #stop_obsData.rename(columns={'sample':test['method']},inplace=True)

In [881]:
def deriveInterpolationMethod(obsData:pd.DataFrame):
    try:
        testResults = []
        methodDict = [{'method':'linear','order':3,'limit_direction':'both'},{'method':'slinear','order':3,'limit_direction':'both'},{'method':'quadratic','order':0,'limit_direction':'both'},{'method':'cubic','order':3,'limit_direction':'both'},{'method':'spline','order':3,'limit_direction':'both'},{'method':'spline','order':5,'limit_direction':'both'},{'method':'polynomial','order':3,'limit_direction':'both'},{'method':'polynomial','order':5,'limit_direction':'both'},{'method':'barycentric','order':5,'limit_direction':'both'}]
        small_methodDict = [{'method':'linear','order':0,'limit_direction':'both'},{'method':'slinear','order':3,'limit_direction':'both'},{'method':'polynomial','order':3,'limit_direction':'both'},{'method':'quadratic','order':0,'limit_direction':'both'}]
        stopKeys = obsData['StopName'].drop_duplicates()
        for x in stopKeys:
            stop_obsData = obsData[obsData['StopName']==x]
            allweek = pd.DataFrame({'obsDt_week':range(1,53)})
            stop_obsData = pd.merge(left=allweek,left_on='obsDt_week',right=stop_obsData,right_on='obsDt_week',how='left')
        #mask
            stop_obsData['mask'] = stop_obsData['relativeAbundance'].interpolate(method='ffill',limit=4,limit_direction='forward')
            stop_obsData['mask'] = stop_obsData['mask'].interpolate(method='bfill',limit=2,limit_direction='backward')
            stop_obsData.loc[stop_obsData['mask'].isna() == True,'relativeAbundance'] = 0
            stop_obsData = stop_obsData[stop_obsData['relativeAbundance'].notna()].drop(columns=['mask'])
            for v in list([0.2,0.25,0.3,0.35]):
                stop_obsData['sample'] = stop_obsData['relativeAbundance'].sample(frac=v,random_state=1)
                if stop_obsData['sample'].count() < 4:
                    chosen = small_methodDict
                else: chosen = methodDict
            else:
                for test in chosen:
                    testName = str(test['method'])
                    stop_obsData[testName] = stop_obsData['sample']
                    stop_obsData[testName] = stop_obsData['sample'].interpolate(method=test['method'],order=test['order'],limit_direction=test['limit_direction']).fillna(stop_obsData['relativeAbundance'])
                    r_sqr = r2_score(y_true=stop_obsData['relativeAbundance'],y_pred=stop_obsData[testName],multioutput='variance_weighted')
                    if r_sqr < 0:
                        continue
                    else: blob = {'StopName':x,'method':test['method'],'order':test['order'],'sampleSize':v,'r_sqr':r_sqr}
                    testResults.append(blob)
        resultFrame = pd.DataFrame.from_dict(testResults)
        resultFrame = resultFrame.groupby(by=['StopName','method','order'])['r_sqr'].mean().reset_index()
        resultFrame.sort_values(by=['StopName','r_sqr'],ascending=False,inplace=True)
        resultFrame.drop_duplicates(subset=['StopName'],keep='first',inplace=True)
                #stop_obsData.rename(columns={'sample':test['method']},inplace=True)
    ##TODO #93 for each stop, return the best interpolation method as determined by the r2 score

    except Exception as problem:
        raise problem
    return resultFrame

In [882]:
gocksweek = deriveInterpolationMethod(gocks)
gocksweek

,StopName,method,order,r_sqr
28,SquakMountain,slinear,3,0.835025
22,MountTeneriffe,slinear,3,0.970703
16,MargaretsWay,slinear,3,0.578065
11,HighSchool,slinear,3,0.755100
6,EastSunsetWay,slinear,3,0.778948
4,ChiricoTrail_PooPooPoint,slinear,3,0.991885


In [806]:
gocksweek['slinear'] = gocksweek['slinear'].interpolate(method='cubic')
gocksweek

,obsDt_week,speciesCode,StopName,relativeAbundance,sample,linear,slinear
0,1,gockin,SquakMountain,0.889662,NaN,NaN,NaN
1,2,gockin,SquakMountain,0.622763,NaN,NaN,NaN
2,3,gockin,SquakMountain,0.266899,0.266899,0.266899,0.266899
3,4,gockin,SquakMountain,2.491054,2.491054,2.491054,2.491054
4,5,gockin,SquakMountain,0.578280,0.578280,0.578280,0.578280
5,6,gockin,SquakMountain,0.355865,NaN,NaN,-1.076708
6,7,gockin,SquakMountain,1.467942,NaN,NaN,-1.780826
7,8,gockin,SquakMountain,0.177932,NaN,NaN,-1.766396
9,10,gockin,SquakMountain,0.177932,NaN,NaN,-0.511184
10,11,gockin,SquakMountain,0.355865,NaN,NaN,0.264955


In [689]:
gocksweek['linear'] = gocksweek['sample'].interpolate(method='linear',order=0,limit_direction='both')
gocksweek['linear3'] = gocksweek['sample'].interpolate(method='linear',order=3,limit_direction='both')
gocksweek['slinear'] = gocksweek['sample'].interpolate(method='slinear',order=0,limit_direction='both')
gocksweek['slinear3'] = gocksweek['sample'].interpolate(method='slinear',order=3,limit_direction='both')
gocksweek['spline3'] = gocksweek['sample'].interpolate(method='spline',order=3).fillna(gocksweek['relativeAbundance'])



TypeError: list indices must be integers or slices, not str

In [677]:
r2score = [(method,r2_score(gocksweek['relativeAbundance'],gocksweek[method],multioutput='variance_weighted')) for method in list(gocksweek)[5:]]
r2score

[('cubic3', 0.3067772412867418),
 ('slinear', 0.8350245301501816),
 ('spline3', 0.1278106831439968)]

In [650]:
max(r2score,key=lambda x:x[1])

('slinear', 0.8143718841492182)

In [435]:
##TODO #90 rewrite stopCovariance to run for a single species at a time --DONE

def stopCovariance(obsData: pd.DataFrame,species: str,StopKey: str,cnx):
    try:
        obsData.sort_values(by=['StopName','obsDt_week'],ascending=True,inplace=True)
        container = []
        resultsContainer = []
        stopKeys = obsData.drop_duplicates(subset=['StopName'])
        for StopName in stopKeys.itertuples():
            stop_obsData = obsData[obsData['speciesCode']==species]
            stop_obsData = stop_obsData[stop_obsData['StopName']==StopName.StopName]
            allweek = pd.DataFrame({'obsDt_week':range(1,53)})
            stop_obsData.drop(columns=['speciesCode','StopName'],inplace=True)
            stop_obsData = pd.merge(left=stop_obsData,right=allweek,left_on='obsDt_week',right_on='obsDt_week',how='outer')
            stop_obsData.set_index('obsDt_week',inplace=True)
            stop_obsData.sort_index(axis='index',ascending=True,inplace=True)
        #TODO #91 write interpolation switches dependent on rarity for stopCovariance()
            stop_obsData['mask'] = stop_obsData['relativeAbundance'].interpolate(method='ffill',limit=5,limit_direction='forward')  #mask, values do not matter
            stop_obsData['mask'] = stop_obsData['mask'].interpolate(method='bfill',limit=2,limit_direction='backward')  #mask, values do not matter
            stop_obsData.loc[stop_obsData['mask'].isna() == True,'relativeAbundance'] = 0
            stop_obsData['fx_relativeAbundance'] = stop_obsData['relativeAbundance'].interpolate(method='linear',limit=5,limit_direction='forward')
            stop_obsData['fx_relativeAbundance'] = stop_obsData['fx_relativeAbundance'].interpolate(method='linear',limit=2,limit_direction='backward')

            stop_obsData.drop(columns=['relativeAbundance','mask'],inplace=True)
            stop_obsData.rename(columns={'fx_relativeAbundance':StopName.StopName},inplace=True)              
            container.append(stop_obsData)
        weeklySpeciesAbd = pd.concat(container,ignore_index=False,axis=1)
        weeklySpeciesAbd = weeklySpeciesAbd.fillna(value=0,axis=0)
        weeklySpeciesAbd = weeklySpeciesAbd.apply(lambda n:(np.log1p(n)),axis=1)
        y_true = weeklySpeciesAbd[StopKey]
    #return related stops
        fit = [(stop,r2_score(y_true,weeklySpeciesAbd[stop])) for stop in list(weeklySpeciesAbd)]
        rSqr = filter(lambda r: (r[1] > 0.25) and (r[1]<1),fit)
        var = [(stop,explained_variance_score(y_true,weeklySpeciesAbd[stop])) for stop in list(weeklySpeciesAbd)]
        expVar = filter(lambda e: (e[1]>0.25) and (e[1]<1),var)
        blob = {'rSquared':list(rSqr),'explVar':list(expVar)}
        log_stopMetrics(speciesCode=species,key=StopKey,testBlob=str(blob),latestUpdate=str(dt.datetime.today()))

    except Exception as ex:
        raise ex
    return {'speciesCode':species,'stopKey':StopKey,'rSquared':list(rSqr),'explVar':list(expVar)}
    #return y_true

In [490]:
amerob = wklyAbd_selectSpecies(cnx=connectDB(),speciesList=['amecro'])

for x in list(amerob['StopName'].drop_duplicates()):
    stopCovariance(obsData=amerob,species='amecro',StopKey=x,cnx=connectDB())

Habitats

In [332]:
def kmeans_habitat(cnx,distinctHabitats: int):
    try:
        data = pd.read_sql(sql='SELECT * FROM FAO_by_locId;',con=cnx)
        data = data.drop(columns=['locName']).set_index('locId')
        data.fillna(0,inplace=True)
    #normalize
        maxValue = data.apply(max,axis=1)
        data = data.apply(lambda x: (x/maxValue[x.index]),axis=0)   #min-max normalizing to smooth in proportionality
    #compute kmeans for each locId
        habitat_kmeans = KMeans(n_clusters=distinctHabitats,init='k-means++')
        habitat_kmeans = habitat_kmeans.fit(data.values)
        clusterLabels = habitat_kmeans.labels_
    #define habitats
        habitatFrame = pd.DataFrame(data=clusterLabels,columns=['clusterLabel'],index=data.index).sort_values(by='clusterLabel').reset_index()
        habitatFrame = pd.merge(left=habitatFrame,left_on='locId',right=data,right_on='locId',how='left')
    except Exception as kmeansExc:
        raise kmeansExc
    return habitatFrame     

In [477]:
def wklyAbd_selectSpecies_locIds(cnx,speciesList: list,locIdList: list):
    cnx=cnx
    try:
        querySpecies = []
        for i in speciesList:
            i = str(i)
            querySpecies.append(i)
        querySpecies = str(querySpecies).strip('[]')
        queryLocs = []
        for i in locIdList:
            i = str(i)
            queryLocs.append(i)
        queryLocs = str(queryLocs).strip('[]')
    #baseline
        baseQuery = f'SELECT speciesCode,obsDt,howMany FROM historicObservations WHERE speciesCode in ({querySpecies});'
        baseline = pd.read_sql(baseQuery,con=cnx,parse_dates=['obsDt'])
        baseline = baseline.assign(obsDt_week=baseline.obsDt.dt.isocalendar().week)
        baselineAvg = baseline.groupby(['speciesCode'])['howMany'].mean()
        #query = f'SELECT speciesCode,FX.locId,StopName,obsDt,howMany FROM historicObservations AS FX  LEFT JOIN closestStop on FX.locId = closestStop.locId WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode in ({querySpecies}) AND FX.locId IN ({queryLocs});'
        query = f'SELECT speciesCode,FX.locId,obsDt,howMany FROM historicObservations AS FX WHERE (SELECT count(distinct(subId)) FROM historicObservations AS QA WHERE QA.comName = FX.comName) > 2 AND FX.speciesCode in ({querySpecies}) AND FX.locId IN ({queryLocs});'
        obsData = pd.read_sql(query,con=cnx,parse_dates=['obsDt'])
        obsData = obsData.assign(obsDt_week=obsData.obsDt.dt.isocalendar().week)
        obsData['howMany'].fillna(1,inplace=True)
        obsData = obsData.groupby(['speciesCode','obsDt_week'])['howMany'].mean().reset_index()
        #maxCount = obsData.groupby(['speciesCode'])['howMany'].max()
        
    #derive relative abundance
        obsData['relativeAbundance'] = obsData.apply(lambda x: (x.howMany/baselineAvg[x.speciesCode]),axis=1)  #baseline relative abd, same process as Fink et. all
        #obsData['relativeAbundance'] = obsData.apply(lambda f: stopQualityMask(f.speciesCode,f.StopName)*f.relativeAbundance,axis=1)  #apply frequency mask
        #avgAnnualAbd = obsData.groupby(['speciesCode'])['relativeAbundance'].mean()
        #obsData['relativeAbundance'] = obsData.apply(lambda n: ((n.relativeAbundance)/(avgAnnualAbd[n.speciesCode])),axis=1)  #normalizing around average relative abundance
        obsData.sort_values(by=['obsDt_week'],ascending=True,inplace=True)
    except db.DatabaseError as dbExc:
        raise f'there was an issue with the database request: {dbExc}'
    except Exception as ex:
        raise ex
    finally: cnx.close()
    return obsData

In [384]:
habs = kmeans_habitat(cnx=connectDB(),distinctHabitats=10)
habs.value_counts(subset=['clusterLabel'])

clusterLabel
6               13
0                9
2                8
1                5
8                5
9                5
3                4
7                3
4                2
5                2
dtype: int64

In [479]:
labelList = habs[habs['clusterLabel']==0]
labelZero = list(labelList['locId'])

In [480]:
result = wklyAbd_selectSpecies_locIds(cnx=connectDB(),speciesList=['gockin'],locIdList=labelZero)
result

,speciesCode,obsDt_week,howMany,relativeAbundance
0,gockin,1,4.666667,0.803470
1,gockin,2,8.000000,1.377377
2,gockin,3,9.250000,1.592593
3,gockin,4,7.000000,1.205205
4,gockin,5,2.000000,0.344344
5,gockin,8,10.000000,1.721722
6,gockin,9,3.000000,0.516517
7,gockin,10,6.000000,1.033033
8,gockin,13,2.000000,0.344344
9,gockin,14,2.500000,0.430430


In [402]:
np.sqrt(((5.837-2.901)**2/3))

1.6951003903407413

In [391]:
gockin = amerob[amerob['speciesCode']=='gockin']

In [544]:
list(amerob[2,])

KeyError: (2,)